In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

# load data

In [2]:
q = '''select * from form where type=3'''
forms_table = redshift_query_read(q, schema='production')

In [3]:
q = '''select
            t.org,
            t.form,
            date_trunc('week', t.date) as week,
            count(t.id) as trans_count,
            sum(t.amount) as trans_vol,
            sum(t.donations_count) as donations_count,
            sum(t.donations_amt) as donations_vol,
            sum(t.purchases_count) as purchases_count,
            sum(t.purchases_amt) as purchases_vol,
            sum(t.registrations_count) as registrations_count,
            sum(t.registrations_amt) as registrations_vol,
            sum(t.events_amt) as events_vol,
            sum(t.events_count) as events_count
        from transactions as t
            left join form as f on f.id=t.form
        where
            t.status='A' and
            f.type=3
        group by date_trunc('week', t.date), t.form, t.org'''
trans = redshift_query_read(q, schema='production')

In [4]:
q = "select * from analyticsp2p_weekly"
df = redshift_query_read(q, schema='public')

### merging

In [5]:
# $41k floor for top 10 volume
top_10_vol_floor = 41000

form_totals = trans.groupby(['form', 'org'])[['trans_vol', 'donations_vol', 'purchases_count', 'purchases_vol', 'registrations_count', 'registrations_vol']].sum().reset_index()
form_totals['is_top_performer'] = form_totals['trans_vol']>top_10_vol_floor
top_performers = form_totals[form_totals['is_top_performer']]['form'].tolist()

In [6]:
len_all = len(form_totals)
bottom_90perc = len_all - len(top_performers)

not_top_10_total_vol = form_totals.sort_values('trans_vol', ascending=False).tail(bottom_90perc)
bottom_performers = not_top_10_total_vol['form'].tolist()

In [7]:
trans['is_top_performer'] = trans['form'].isin(top_performers)
df['is_top_performer'] = df['form'].isin(top_performers)

trans['is_bottom_performer'] = trans['form'].isin(bottom_performers)
df['is_bottom_performer'] = df['form'].isin(bottom_performers)
form_totals['is_bottom_performer'] = form_totals['form'].isin(bottom_performers)

In [8]:
def form_cat(form):
    if form in forms_table['id'].tolist():
        return forms_table[forms_table['id']==form]['categorization'].iloc[0]
    return 0

form_totals['category'] = form_totals['form'].apply(form_cat)

In [9]:
def form_isvirtual(form):
    if form in forms_table['id'].tolist():
        return forms_table[forms_table['id']==form]['isvirtual'].iloc[0]
    return 0

form_totals['isvirtual'] = form_totals['form'].apply(form_isvirtual)

In [10]:
form_totals = form_totals.merge(df.groupby('form')['teams_count'].max().reset_index(), on='form')

# analysis

## isvirtual

In [13]:
form_totals.groupby('isvirtual')[['trans_vol', 'donations_vol', 'registrations_vol', 'registrations_count', 'is_top_performer', 'is_bottom_performer']].agg(['median', 'mean']).transpose()

isvirtual                              0             1
trans_vol           median   4991.750000   7189.240000
                    mean    18351.280086  20675.059533
donations_vol       median   4226.920000   6680.000000
                    mean    16494.766453  19657.608879
registrations_vol   median      0.000000      0.000000
                    mean     1286.218519    439.115265
registrations_count median     16.000000     15.000000
                    mean       93.611877     58.190031
is_top_performer    median      0.000000      0.000000
                    mean        0.102746      0.118380
is_bottom_performer median      1.000000      1.000000
                    mean        0.897254      0.881620

## category

In [18]:
form_totals.groupby('category')['form'].count()

category
0     6136
1      346
2       60
3      115
4       72
5      245
6      449
7       69
8      489
9      201
10      11
11      73
12      15
13      18
14      22
15      49
Name: form, dtype: int64

In [15]:
form_totals.groupby('category')[['trans_vol', 'donations_vol', 'registrations_vol', 'registrations_count', 'is_top_performer', 'is_bottom_performer']].agg(['median', 'mean'])

trans_vol               donations_vol                \
             median          mean        median          mean   
category                                                        
0          4883.200  16204.147513      4223.000  14603.645104   
1          7756.375  18573.435954      6651.995  16925.160491   
2         10484.315  20720.620500     10237.285  19552.847167   
3          7524.300  29116.151130      6304.750  27442.394435   
4          8488.600  37575.469306      5877.345  28942.541528   
5         12579.020  34858.804000      9888.000  29034.836939   
6          6750.000  26197.122272      4883.000  22599.514744   
7          2375.000  12545.409130      2375.000  12304.131884   
8          2056.000  19388.929346      1950.000  18846.990859   
9          7239.330  30337.870050      6594.250  28527.324328   
10         6874.800  16953.298182      6720.000  16640.290909   
11         4408.640  12564.364932      4395.000  11918.335068   
12         9935.000  10154.762000       300.000   1482.920000   
13         3493.515  32505.371667      3425.000  31830.748333   
14          790.000  33166.673182         5.000  32139.941364   
15         2520.000  38278.466327      2451.750  36413.386327   

         registrations_vol              registrations_count              \
                    median         mean              median        mean   
category                                                                  
0                      0.0  1219.032044                16.0   92.025587   
1                      0.0   894.187023                41.5  112.014451   
2                      0.0   794.150000                22.5   72.066667   
3                      0.0   844.808696                23.0   85.113043   
4                      0.0  2397.916667                 3.5   82.458333   
5                   1330.0  4755.946000                88.0  272.371429   
6                      0.0  2226.711047                34.0   96.267261   
7                      0.0    22.028986                 1.0   11.492754   
8                      0.0    98.267894                 1.0   40.609407   
9                      0.0   582.238806                 2.0   54.427861   
10                     0.0     0.000000                 1.0    6.909091   
11                     0.0   199.547945                 4.0   36.383562   
12                     0.0     0.000000                 0.0    0.266667   
13                     0.0    59.166667                 0.0   11.555556   
14                     0.0   403.636364                 3.5   60.272727   
15                     0.0   628.418367                 6.0   58.571429   

         is_top_performer           is_bottom_performer            
                   median      mean              median      mean  
category                                                           
0                     0.0  0.092405                 1.0  0.907595  
1                     0.0  0.118497                 1.0  0.881503  
2                     0.0  0.083333                 1.0  0.916667  
3                     0.0  0.139130                 1.0  0.860870  
4                     0.0  0.277778                 1.0  0.722222  
5                     0.0  0.236735                 1.0  0.763265  
6                     0.0  0.149220                 1.0  0.850780  
7                     0.0  0.086957                 1.0  0.913043  
8                     0.0  0.067485                 1.0  0.932515  
9                     0.0  0.154229                 1.0  0.845771  
10                    0.0  0.181818                 1.0  0.818182  
11                    0.0  0.027397                 1.0  0.972603  
12                    0.0  0.000000                 1.0  1.000000  
13                    0.0  0.055556                 1.0  0.944444  
14                    0.0  0.136364                 1.0  0.863636  
15                    0.0  0.265306                 1.0  0.734694